1. import toghter.py and call the next_block to completely load the entire dataset and all , train the model and then use it.
2. import load_model.py and call the next_block to just use the saved model and directly use the model.predict but it takes a very large amount of time to output prediction..... the time more or less same as that of the training of the model....but training the model is at backend so preferably use the toghter.py file .
3. The PROJECT folder has the two directories namely Audio_text and Only_text for multimodal and unimodal sentiment analysis respectively .
4. Both the directories have thier own Datasets and Payments folder which contains the necessary datasets and (html+css+js) files requied.
5. The paths of the datasets is given appropriately in the code only one needs to upload the PROJECT folder directly onto the MyDrive section.
6.   

In [ ]:
!pip install keras
!pip install nltk
!pip install math
!pip install flask-ngrok
!pip install SpeechRecognition
!apt install libasound2-dev portaudio19-dev
!pip install PyAudio
!pip install librosa
!pip install wavio

from flask import Flask, render_template
from PIL import Image
import io
import base64

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#RUN THIS BLOCK OF THE NOTEBOOK TO WORK WITH OUR AUDIO AND  TEXT BASED SENTIMENTAL ANALYSIS

from flask_ngrok import run_with_ngrok
from subprocess import run, PIPE
from flask import Flask,render_template,logging,request,redirect,url_for
import threading
from google.colab import files
from scipy.io import wavfile
%cd /content/drive/MyDrive/PROJECT/Audio_text/Payment
#import load_model
import togther
import librosa
import numpy as np
import time
import pyaudio
import speech_recognition as sr

#(The setup for flask backend)
app=Flask(__name__)
run_with_ngrok(app)
app.config['UPLOAD_FOLDER']=True
app. config["SECRET_KEY"] =True
important=""


#(The google speech to text converter on received audio file)
def STT(file):
  def callback(recognizer , audio):
    try:
        input_string=recognizer.recognize_google(audio,language="en-SG")
        global important
        important=input_string
    except:
        print("Opps didn't catch")
  r=sr.Recognizer()
  m=sr.AudioFile(file)
  with m as source:
        r.dynamic_energy_treshold=True
        r.adjust_for_ambient_noise(source,duration=5)
        time.sleep(0.5)
  stop_listening=r.listen_in_background(m,callback)
  for _ in range(8):time.sleep(0.1) 
  stop_listening()
  for i in range(5):time.sleep(0.1)


#(This is the route to the homepage of the web display)
@app.route("/",methods=['GET'])
def index():
    return render_template('index.html')

#(this method is called as a response to the submission of the form in the html code)
@app.route("/output", methods=['POST'])
def upload_file():
    file=request.files['file']
    if file.filename != '':
        file.save(file.filename)
        y,sr=librosa.load(file.filename)
        y = (np.iinfo(np.int32).max * (y/np.abs(y).max())).astype(np.int32)
        wavfile.write(file.filename, sr, y)
        STT(file.filename)
        #emotion=load_model.next_block(file.filename,important)
        emotion=togther.next_block(file.filename,important)
        im = Image.open("/content/drive/MyDrive/PROJECT/Audio_text/Payment/static/"+emotion+".jfif")
        data = io.BytesIO()
        im.save(data, "JPEG")
        encoded_img_data = base64.b64encode(data.getvalue())
        return render_template('index.html',transcript=important,emotion=emotion,image=encoded_img_data.decode('utf-8'))
    return redirect(url_for('index',transcript="important"))

#(continously run the backend until we manually terminate the code)
threading.Thread(target=app.run()).start()

In [ ]:
#RUN THIS BLOCK OF THE NOTEBOOK TO WORK WITH OUR ONLY THE TEXT BASED SENTIMENTAL ANALYSIS

from flask_ngrok import run_with_ngrok
from subprocess import run, PIPE
from flask import Flask,render_template,logging,request,redirect,url_for
import threading
from google.colab import files
%cd /content/drive/MyDrive/PROJECT/Only_text/Payment

import mylib
app=Flask(__name__)
run_with_ngrok(app)
app.config['UPLOAD_FOLDER']=True
app. config["SECRET_KEY"] =True
important=""


def STT(file):
  import time
  import pyaudio
  import speech_recognition as sr
  def callback(recognizer , audio):
    try:
        input_string=recognizer.recognize_google(audio,language="en-SG")
        global important
        important=input_string
    except:
        print("Opps didn't catch")
  r=sr.Recognizer()
  m=sr.AudioFile(file)
  with m as source:
        r.dynamic_energy_treshold=True
        r.adjust_for_ambient_noise(source,duration=5)
        time.sleep(0.5)
  stop_listening=r.listen_in_background(m,callback)
  for _ in range(8):time.sleep(0.1) 
  stop_listening()
  for i in range(5):time.sleep(0.1)


@app.route("/",methods=['GET'])
def index():
    return render_template('index.html')


@app.route("/output", methods=['POST'])
def upload_file():
    file=request.files['file']
    if file.filename != '':
        file.save(file.filename)
        STT(file.filename)
        emotion=mylib.next_block(important)
        im = Image.open("/content/drive/MyDrive/PROJECT/Only_text/Payment/static/"+emotion+".jfif")
        data = io.BytesIO()
        im.save(data, "JPEG")
        encoded_img_data = base64.b64encode(data.getvalue())
        return render_template('index.html',transcript=important,emotion=emotion,image=encoded_img_data.decode('utf-8'))
    return redirect(url_for('index',transcript="important"))


threading.Thread(target=app.run()).start()